# Othello Board

In [1]:
          
#############################################################################################################################################

class OthelloBoardArray():
   
    def __init__ (self):

        '''
        Board is like:
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']                                
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
        '''

        self.Board = [[' '] * 8 for i in range(8)]
        self.Board[3][3], self.Board[4][4] = 'W', 'W'
        self.Board[3][4], self.Board[4][3] = 'B', 'B'

In [2]:
class OthelloBoardArray(OthelloBoardArray):        
    #Creatin a temporary board.
    def MakeTempBoard(self, GivenBoard):
        TempBoard = [[' '] * 8 for i in range(8)]
        for i in range(8):
            for j in range(8):
                TempBoard[i][j] = GivenBoard[i][j]

        return TempBoard
      

In [3]:
class OthelloBoardArray(OthelloBoardArray):             
    #Check if the move is legal and return the cells to be filled if it so.
    def isLegalMove(self, Disc, CellLocation, GivenBoard):
        
        '''
        x_loc = CellLocation[0]
        y_loc = CellLocation[1]
        
        if GivenBoard[x_loc][y_loc] == 'W' or GivenBoard[x_loc][y_loc] == 'B':
            print(x_loc, y_loc)
            return 'False'
        '''   
        
        # Setting the discs
        if Disc == 'W':
            OpponentDisc = 'B'
        else:
            OpponentDisc = 'W'    

        # Creating the board directions
        EightDirections = []
        
        for i in range(-1,2):
            for j in range(-1,2):
                EightDirections.append([i,j])

        EightDirections.remove([0,0])

        #Creating an array to store the locations of cells to be flipped.
        CellsToBeFlipped = []

        #Checking all the Directions
        for Direction in EightDirections:
            
            XDirection = Direction[0]
            YDirection = Direction[1]
            x = CellLocation[0]
            y = CellLocation[1]

            x = x + XDirection
            y = y + YDirection

            TempCellStorage = []

            while(((x in range(8)) and (y in range(8))) and GivenBoard[x][y]==OpponentDisc):
                TempCellStorage.append([x,y])

                x = x + XDirection
                y = y + YDirection

                if not((x in range(8)) and (y in range(8))) or (GivenBoard[x][y]==' '):
                    TempCellStorage.clear()
                    break

                if (GivenBoard[x][y]==Disc):
                    break


            CellsToBeFlipped.extend(TempCellStorage)

        if len(CellsToBeFlipped) == 0:
            return 'False'
        else:
            return CellsToBeFlipped

In [4]:
class OthelloBoardArray(OthelloBoardArray): 
    #Find all possible legal moves.
    def FindAllLegalMoves(self, Disc, GivenBoard):
        
        LegalMoves = []
        
        for i in range(8):
            for j in range(8):
                if (GivenBoard[i][j] == ' ') and (self.isLegalMove(Disc, [i,j], GivenBoard) != 'False'):
                    LegalMoves.append([i,j])

        return LegalMoves

In [5]:
class OthelloBoardArray(OthelloBoardArray): 
    #Calculate the score.
    def CalculateScore(self, GivenBoard):

        WhiteScore, BlackScore = 0,0

        for i in range(8):
            for j in range(8):
                if (GivenBoard[i][j]=='W'):
                    WhiteScore += 1
                if(GivenBoard[i][j]=='B'):
                    BlackScore += 1

        return {'W': WhiteScore,'B': BlackScore}

In [6]:
class OthelloBoardArray(OthelloBoardArray): 
    #Update the Board.
    def UpdateBoard(self, GivenBoard, CellsToBeFlipped):

        for Cell in CellsToBeFlipped:
            #print("Cell is: ", Cell)
            x = Cell[0]
            y = Cell[1]
            if GivenBoard[x][y] == 'W':
                GivenBoard[x][y] = 'B'
            else:
                GivenBoard[x][y] = 'W'

        return  GivenBoard

In [7]:
class OthelloBoardArray(OthelloBoardArray): 
    #Check if the move is a corner move.
    def isCornerMove(self, x, y):
        return (x == 0 or x == 7) and (y == 0 or y == 7)

In [8]:
class OthelloBoardArray(OthelloBoardArray): 
    #Choose the move for the computer.
    def ChooseComputerMove(self, Disc, GivenBoard):
        # Setting the discs
        if Disc == 'W':
            OpponentDisc = 'B'
        else:
            OpponentDisc = 'W'

        LegalMoves = self.FindAllLegalMoves(OpponentDisc, GivenBoard)
        
        #print("LegalMoves: ", LegalMoves)

        MaxScore = 0
        MaxMove = [-1,-1]

        for Move in LegalMoves:
            x = Move[0]
            y = Move[1]

            if self.isCornerMove(x, y):
                MaxMove = [x,y]
                break

            TempBoard = self.MakeTempBoard(GivenBoard)
            TempBoard[x][y] = OpponentDisc
            CellsToBeFlipped = self.isLegalMove(OpponentDisc, Move, TempBoard)
            #print("CellsToBeFlipped: ", CellsToBeFlipped)
            #return [-1,-1]
            if CellsToBeFlipped != 'False':
                TempBoard = self.UpdateBoard(TempBoard, CellsToBeFlipped)
                Score = self.CalculateScore(TempBoard)
            
                if Score[OpponentDisc] > MaxScore:
                    MaxScore = Score[OpponentDisc]
                    MaxMove = Move

        return MaxMove

# PPO

In [9]:
import glob
import os
import time

from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker

import gymnasium as gym
import pygame
import sys
import random
import numpy as np

import stable_baselines3
import pettingzoo

from stable_baselines3.common.utils import safe_mean

In [ ]:
"""
Based on - 
Author: Elliot (https://github.com/elliottower)
URL: https://github.com/Farama-Foundation/PettingZoo/blob/master/tutorials/SB3/connect_four/sb3_connect_four_action_mask.py
"""

In [10]:
class SB3ActionMaskWrapper(pettingzoo.utils.BaseWrapper):
    """Wrapper to allow PettingZoo environments to be used with SB3 illegal action masking."""

    def reset(self, seed=None, options=None):
        """Gymnasium-like reset function which assigns obs/action spaces to be the same for each agent.

        This is required as SB3 is designed for single-agent RL and doesn't expect obs/action spaces to be functions
        """
        super().reset(seed, options)

        # Strip the action mask out from the observation space
        self.observation_space = super().observation_space(self.possible_agents[0])[
            "observation"
        ]
        self.action_space = super().action_space(self.possible_agents[0])

        # Return initial observation, info (PettingZoo AEC envs do not by default)
        return self.observe(self.agent_selection), {}
        
    def step(self, action):
        """Gymnasium-like step function, returning observation, reward, termination, truncation, info."""
        super().step(action)
        agent = self.agent_selection
        if (self.terminations['player_B'] == True):
            return super().last()
        else:
            obs = self.observe(self.agent_selection)
            if agent[-1] == 'B':
                PlayerDisc = 'W'
                ComputerDisc = 'B'
            else:
                PlayerDisc = 'B'
                ComputerDisc = 'W'
            OthelloBoard = self.obs_to_board(obs)
            OthelloGame = OthelloBoardArray()
            
            ComputerMove = OthelloGame.ChooseComputerMove(PlayerDisc, OthelloBoard)
            comp_action = self.cell_to_action(ComputerMove)
            super().step(comp_action)
            return super().last()
        
        return super().last()

    def observe(self, agent):
        """Return only raw observation, removing action mask."""
        return super().observe(agent)["observation"]

    def action_mask(self):
        """Separate function used in order to access the action mask."""
        return super().observe(self.agent_selection)["action_mask"]

In [11]:
def mask_fn(env):
    return env.action_mask()

In [12]:
def train_action_mask(steps=10000, seed=8):
    """Train a single model to play as each agent in a zero-sum game environment using invalid action masking."""
    
    # Custom wrapper to convert PettingZoo envs to work with SB3 action masking
    %run Othello.ipynb
    %run MaskablePPO.ipynb
    env = OthelloEnv()
    env = SB3ActionMaskWrapper(env)

    env.reset(seed=seed)  # Must call reset() in order to re-define the spaces

    env = ActionMasker(env, mask_fn) 
    model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=1)
    model.set_random_seed(seed)

    model.learn(total_timesteps=steps)

    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.\n")

    env.close()


In [17]:
train_action_mask(steps=int(1e6))

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 29.4     |
|    ep_rew_mean     | -503     |
| time/              |          |
|    fps             | 439      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 28.6         |
|    ep_rew_mean          | -466         |
| time/                   |              |
|    fps                  | 261          |
|    iterations           | 2            |
|    time_elapsed         | 15           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0017903633 |
|    clip_fraction        | 0.000146     |
|    clip_range           | 0.2          |
|    e

In [19]:
%run MaskablePPO.ipynb
latest_policy = max(glob.glob(f"{'othello_PPO'}*.zip"), key=os.path.getctime)
model = MaskablePPO.load(path=latest_policy)
model.set_random_seed(8)
validate(100, model)

0.95